# Tema 3 - Exercitii `ipywidgets`

In [1]:
import ipywidgets as widgets
print(f'IPywidgets version: {widgets.__version__}')

import numpy as np
print(f'NumPy version: {np.__version__}')

import matplotlib.pyplot as plt
import matplotlib
print(f'Matplotlib version: {matplotlib.__version__}')

IPywidgets version: 7.6.3
NumPy version: 1.19.2
Matplotlib version: 3.3.4


In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual

### Problema 1
Definiti o functie polinomiala de gradul 3, $f:\mathbb{R} \rightarrow \mathbb{R}$, cu coeficienti constanti prestabiliti. Aplicati algoritmul gradient descent pentru a vedea cum evolueaza cautarea minimului. Folositi minim doua controale ipywidgets: unul pentru pozitia initiala a lui $x$, altul pentru coeficientul $\alpha>0$ cu care se inmulteste gradientul. Gradientul va fi calculat analitic de voi sau folosind biblioteca [autograd](https://github.com/HIPS/autograd). 
Modificarea facuta prin metoda gradient descent este:
$$
x = x - \alpha \cdot f'(x)
$$
Se vor efectua minim 10 iteratii (optional: numarul de iteratii poate fi dat printr-un control ipywidgets), se vor marca pe grafic pozitiile succesive, in mod convenabil. 

In [3]:
f = lambda x: 3*x**3 + (-2)*x**2 + 2*x + 3
grad = lambda x: 9*x**2 + (-4)*x + 2

xf = np.linspace(-10, 10, 100)
yf = f(xf)

def sgd_plot(xf, yf, res=None, title = 'Functia $f(x)=3x^3 -2x^2 + 2x + 3$'):
    _, ax = plt.subplots(1, 1, figsize=(10, 5))
    ax.plot(xf, yf)
    if res is not None:
        ax.plot(res, [f(x) for x in res], 'o')
    ax.set_title(title)
    ax.grid(axis='both')
#     ax.axhline(y=0, color='k')
#     ax.axvline(x=0, color='k')
    plt.show()

def sgd(x=10, alpha=0.001, epochs=10):
    res = [x]
    for _ in range(epochs):
        x -= alpha * grad(x)
        res += [x]
    return res

def gradient_descent(x=10, alpha=0.001, epochs=10) -> None:
    res = sgd(x, alpha, epochs)
    sgd_plot(xf, yf, res, title=f'$f(x) = 3x^3 -2x^2 + 2x + 3$, start x={x}, alpha={alpha}')

interact(gradient_descent, x=(-10,10), alpha=(1e-3,5e-3,0.0001), epochs=(10,100));

interactive(children=(IntSlider(value=10, description='x', max=10, min=-10), FloatSlider(value=0.001, descript…

### Problema 2

Generati o lista de $n=100$ de perechi de valori $\{x_i, y_i\}_{i=0,n-1}$ in intervalul [-20, 10), afisati aceste valori pe un grafic, impreuna cu o dreapta definita de o functie liniara $y=a \cdot x+b$. Intr-un alt plot afisati, ca histograma, distanta dintre punctele de coordonate $(x_i, y_i)$ si punctele de intersectie ale verticalelor duse prin $x_i$ cu dreapta data, $\hat{y}_i$. Dreapta trebuie sa fie controlabila din widgets, prin cei doi coeficienti $a$ si $b$. Constatati modificarea histogramei in functie de pozitia dreptei si afisati mean squared error: $$MSE=\frac{1}{n} \cdot \sum_{i=0}^{n-1} (y_i - (a\cdot x_i + b))^2$$.
*Indicatii:*
1. Pentru generare de valori distribuite uniform in intervalul [0, 1) puteti folosi functia [numpy.random.uniform](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.uniform.html) si sa faceti inmultire si adunare in mod convenabil.
1. Puteti opta sa returnati cele $n$ puncte sub forma `vector_x`, `vector_y`.

In [8]:
def line(a=1, b=1):
    x_rand = np.random.uniform(-20, 10, 100)
    y_rand = np.random.uniform(-20, 10, 100)
    range_x = np.linspace(-20, 10, 100)
    values_y = a * range_x + b
    
    predicted_y = a * x_rand + b
    vertical_distances = y_rand - predicted_y
#     print(f'Vertical distances: \n {vertical_distances}')
    print(f'Distanta verticala minima este: {min(vertical_distances)}')
    print(f'Distanta verticala maxima este: {max(vertical_distances)}')
    plt.figure(figsize=(10,5))
    plt.xlabel('x')
    plt.ylabel('y = ax + b')
    plt.plot(range_x, values_y, 'r', x_rand, y_rand, 'o')
    plt.grid(axis='both')
    plt.axhline(y=0, color='k')
    plt.axvline(x=0, color='k')
    plt.show()
    
    plt.hist(vertical_distances, bins=20)
    plt.xlabel('Vertical Distance')
    plt.ylabel('Number of appearances')
    plt.show()

    MSE = np.square(np.subtract(y_rand,predicted_y)).mean()
    print(f'MSE = {MSE}')
    
interact(line, a=(-10, 10.0), b=(-10, 10.0));

interactive(children=(FloatSlider(value=1.0, description='a', max=10.0, min=-10.0), FloatSlider(value=1.0, des…

### Problema 3
Incarcati fisierul `iris.data` din [setul de date iris](https://archive.ics.uci.edu/ml/datasets/iris). In functie de alegerile exprimate de un utilizator, afisati intr-un grafic 2D coloanele numerice alese (de exemplu, coloana 0 si coloana 2). Numele coloanelor se afla in fisierul iris.names.

*Indicatii/optiuni*:
* Incarcarea de date se poate face cu numpy, functia [loadtxt](https://docs.scipy.org/doc/numpy/reference/generated/numpy.loadtxt.html). Specificati faptul ca se sare peste prima linie din fisier (header). Alternativ, puteti folosi [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).
* Pentru cele doua alegeri puteti sa instantiati doua obiecte [Dropdown](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#Dropdown) sau [Select](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#Select).

In [68]:
data = np.loadtxt("iris.data", delimiter=',', usecols=range(4))

sepal_dropdown = widgets.Dropdown(
    options=['sepal length in cm', 'sepal width in cm'],
    description='Sepal: ',
    disabled=False,
)
petal_dropdown = widgets.Dropdown(
    options=['petal length in cm', 'petal width in cm'],
    description='Petal: ',
    disabled=False,
)

def plot_it(sepal_dropdown, petal_dropdown):
    plt.title("Iris Data Set")
    plt.xlabel('Data number')
    plt.ylabel('Centimeters')
    if sepal_dropdown == "sepal length in cm":
        col1 = data[:, 0]
        plt.plot(col1, 'r*', label='Sepal length in cm')
    else:
        col2 = data[:, 1]
        plt.plot(col2, 'gv', label='Sepal width in cm')
    if petal_dropdown == "petal length in cm":
        col3 = data[:, 2]
        plt.plot(col3, 'b^', label='Petal length in cm')
    else:
        col4 = data[:, 3]
        plt.plot(col4, 'c+', label='Petal width in cm')
    plt.legend()
    
interact(plot_it, sepal_dropdown=sepal_dropdown, petal_dropdown=petal_dropdown);

interactive(children=(Dropdown(description='Sepal: ', options=('sepal length in cm', 'sepal width in cm'), val…

### Problema 4
Generati $n$ perechi de puncte aleatoare, folosind o functie $f:\mathbb{R} \rightarrow \mathbb{R}$ de alease e voi (de exemplu: functie polinomiala + zgomot aleator adaugat). Alegeti 5 metode de interpolare din [scipy.interpolate](https://docs.scipy.org/doc/scipy/reference/interpolate.html) si reprezentati grafic valorile interpolate. Folositi controale ipywidgets cel putin pentru alegerea lui $n$ si a metodei de interpolare aleasa.

In [86]:
from scipy import interpolate
from scipy.interpolate import barycentric_interpolate, pchip_interpolate

interpolation_dropdown = widgets.Dropdown(
    options=["1-D Interpolation", "Barycentric Interpolation", "Pchip Interpolation"],
    description="Interpolation Method: ",
    disabled=False,
)
def funct(n, interpolation_dropdown):
    range_x = np.linspace(-10,10,n)
    values_y = 2*range_x**3 + 2*range_x**2 - 3*range_x + 7
    
    plt.figure(figsize=(10,5))
    plt.xlabel('x')
    plt.ylabel('y = $-3x^2 + 6x -7$')
    plt.grid(axis='both')
    plt.axhline(y=0, color='k')
    plt.axvline(x=0, color='k')
    if interpolation_dropdown == '1-D Interpolation':
        f = interpolate.interp1d(range_x, values_y)
        xnew = np.arange(-10,10,0.5)
        ynew = f(xnew)
        plt.plot(range_x, values_y, 'r-', label='observation')
        plt.plot(xnew, ynew, 'ko', label='1-D Interpolation')
        plt.legend()
    elif interpolation_dropdown == "Barycentric Interpolation":
        x = np.linspace(min(range_x), max(range_x), num=10)
        y = barycentric_interpolate(range_x, values_y, x)
        plt.plot(range_x, values_y, 'r-', label="observation")
        plt.plot(x, y, 'bo', label='Barycentric Interpolation')
        plt.legend()
    elif interpolation_dropdown == "Pchip Interpolation":
        x = np.linspace(min(range_x), max(range_x), num=10)
        y = pchip_interpolate(range_x, values_y, x)
        plt.plot(range_x, values_y, 'r-', label='observation')
        plt.plot(x, y, 'go', label='pchip Interpolation')
        plt.legend()

    
interact(funct, n=(10, 100), interpolation_dropdown=interpolation_dropdown);


interactive(children=(IntSlider(value=55, description='n', min=10), Dropdown(description='Interpolation Method…